In [1]:
import ltcode as lt
from ltcode import LOG
import dnaIO as dna
fileName = "data/hat-768x512.png"
outFile = "data/hat-768x512-out.png"

hashInit = dna.compute_file_hash(fileName)
LOG.Basic("INIT", "File Hash: %s" % hashInit)

dnaConv = dna.DNAConverter2()
byteConv = dna.AssembleConverter2()

[00:20:06] [INIT        ] File Hash: 5e9f48bd613eacfb6252c77ae7306a73


In [2]:
# 载入并切分文件
# 喷泉码比例
lt_ratio = 2.0
fileBlocks, fileBlocksN, dropQuantity, fileSize = lt.load_files(fileName, 48, lt_ratio)

# 生成喷泉码
fileSymbols = lt.create_symbols(fileBlocks, dropQuantity)

[00:20:06] [INIT        ] Filesize: 502888 bytes
[00:20:06] [INIT        ] Blocks: 1965 -> 3930 * 256 bytes
[00:20:06] [ENCODE      ] Start Encode
[00:20:06] [LT-ENCODE   ] Generating graph...
[00:20:06] [LT-ENCODE   ] Ready for encoding.
[00:20:06] [LT-ENCODE   ] Correctly dropped 3930 symbols (packet size=65536)
[00:20:06] [ENCODE      ] End Encode


In [3]:
# 生成拼接比特流
bitsArr = byteConv.convert_bytes_form_symbols(fileSymbols, fileSize, blocksN=fileBlocksN)

[00:20:06] [ASSEM-ENCODE] Convert Symbol->Bytes Start
[00:20:06] [ASSEM-ENCODE] Convert Symbol->Bytes Finish


In [4]:
# 生成DNAArr
dnaArr = dna.convert_dnaArr_from_bitArr(bitsArr, conv=dnaConv)

[00:20:06] [DNA-ENCODE  ] Convert Bytes->DNA Start use: Same2
[00:20:09] [DNA-ENCODE  ] Convert Bytes->DNA Finish Count: 5033152/3930=1280
[00:20:09] [DNA-ENCODE  ] Convert Bytes->DNA CGRatio: 0.468 0.473 0.474 0.475 0.479


In [5]:
# 进行模拟损毁
# 丢失率
lost = 0.05
# 替换率
replace = 0.01
# PCR扩增率
pcr = 10
dnaArrOut = dna.destory_DNAs(dnaArr, lost, pcr, replace)

[00:20:10] [DESTORY     ] Destroy DNA 3930 --PCR-> 39300 --Destory(0.050000)-> 37400


In [6]:
# 使用DNA进行恢复为比特流
bitsArrOut = dna.convert_bitArr_from_DNAs(dnaArrOut, conv=dnaConv)

[00:20:10] [DNA-DECODE  ] Convert DNA->Bytes Start use: Same2


KeyboardInterrupt: 

In [ ]:
# 从比特流恢复到喷泉码
symbolsOut, fileSizeOut, fileBlocksNOut = byteConv.convert_symbols_from_bitArray(bitsArrOut)

In [ ]:
# 使用喷泉码进行恢复
recoveredBlocks, recoverdN = lt.decode_from_symbols(symbolsOut, fileBlocksNOut)

# 导出恢复的文件
with open(outFile, "wb") as file:
    lt.blocks_write(recoveredBlocks, file, fileSizeOut)

hashOut = dna.compute_file_hash(outFile)
LOG.Basic("FINISH", "OutFile Hash: %s" % hashOut)
LOG.Basic("FINISH", "File Convert %s" % ("Success" if hashOut == hashInit else "Failed"))